In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
def load_env_to_dict(file_path):
    env_dict = {}
    with open(file_path, "r") as file:
        for line in file:
            # Remove whitespace and ignore comments or empty lines
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            # Split the line into key and value
            key, value = line.split("=", 1)
            env_dict[key.strip()] = value.strip()
    return env_dict

In [11]:
file_path = "/mnt/Exdisk/git-cuongpiger/secret/work/vngcloud/ai-platform/env"
env_variables = load_env_to_dict(file_path)

# Import dependencies


In [12]:
import os

from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, BaseMessage
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.runnables import chain
from langchain_core.utils.function_calling import tool_example_to_messages
from pydantic import BaseModel, Field
from enum import Enum
from langgraph.graph.message import add_messages
from langchain_core.messages import trim_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import ChatOllama


from IPython.display import display, Markdown, Latex

from typing_extensions import Annotated, TypedDict
from typing import List, Optional, Sequence

# Configure Google credentials

- **NOTE**: Remember change the `GOOGLE_APPLICATION_CREDENTIALS` to the path of your own Google credentials file.


In [13]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/home/cuongdm/git-cuongpiger/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

# Model

In [15]:
# model = ChatVertexAI(temperature=0, model="gemini-1.5-flash")

# llm = ChatVertexAI(temperature=0, model="gemini-1.5-flash")
model = ChatOllama(
    temperature=0, model="llama3.3:70b", base_url=env_variables["OLLAMA_SERVER_URL"]
)

In [16]:
model.invoke([HumanMessage(content="Xin chào, tao là Tứ")])

AIMessage(content='Xin chào Tứ! Rất vui được gặp bạn. Tôi có thể giúp gì cho bạn hôm nay?', additional_kwargs={}, response_metadata={'model': 'llama3.3:70b', 'created_at': '2025-01-23T10:40:11.161120876Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9193000088, 'load_duration': 6933688860, 'prompt_eval_count': 20, 'prompt_eval_duration': 284000000, 'eval_count': 25, 'eval_duration': 1973000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-3dc81514-4aa2-49b7-8c10-22b135200b7f-0', usage_metadata={'input_tokens': 20, 'output_tokens': 25, 'total_tokens': 45})

In [17]:
model.invoke(
    [HumanMessage(content="Nhiều khi tao muốn một lần nói ra hết tất cả thay vì.")]
)

AIMessage(content='Có vẻ như bạn đang cảm thấy có nhiều điều muốn nói, nhưng lại không biết bắt đầu từ đâu hoặc cảm thấy khó khăn khi thể hiện tất cả những suy nghĩ và cảm xúc của mình. Điều này hoàn toàn bình thường, và có nhiều cách để bạn có thể giải quyết tình huống này.\n\nDưới đây là một số gợi ý có thể giúp bạn:\n\n1. **Viết nhật ký**: Viết ra tất cả những gì bạn đang nghĩ và cảm nhận vào một cuốn nhật ký. Đây có thể là một cách tuyệt vời để bạn xử lý và hiểu rõ hơn về cảm xúc của mình mà không cần phải lo lắng về việc phải nói tất cả ra cùng một lúc.\n\n2. **Nói chuyện với người tin cậy**: Chọn một người bạn tin tưởng, như một người bạn thân, thành viên gia đình hoặc thậm chí là một chuyên gia tư vấn. Nói chuyện với họ có thể giúp bạn cảm thấy nhẹ nhõm hơn và nhận được sự hỗ trợ cần thiết.\n\n3. **Tập trung vào một vấn đề tại một thời điểm**: Thay vì cố gắng nói tất cả mọi thứ cùng một lúc, hãy tập trung vào một vấn đề hoặc chủ đề cụ thể mỗi lần. Điều này có thể giúp cuộc trò c

In [18]:
model.invoke(
    [
        HumanMessage(content="VKS là gì?"),
        AIMessage(
            content="VKS là viết tắt của VngCloud Kubernetes Service, một dịch vụ của VngCloud nhằm cung cấp các cụm Kubernetes đến khách hàng. VKS cung cấp các tính năng như fully managed Kubernetes, autoscaling, monitoring, logging, và nhiều tính năng khác."
        ),
        HumanMessage(content="Các tính năng của VKS là gì?"),
    ]
)

AIMessage(content='VKS (VngCloud Kubernetes Service) cung cấp các tính năng sau:\n\n1. **Fully Managed Kubernetes**: VKS quản lý toàn bộ quá trình tạo lập, cấu hình và vận hành cụm Kubernetes, giúp khách hàng tập trung vào việc phát triển ứng dụng mà không cần phải lo lắng về việc quản lý hạ tầng.\n2. **Autoscaling**: Tự động điều chỉnh số lượng node trong cụm Kubernetes dựa trên nhu cầu thực tế, giúp tối ưu hóa tài nguyên và giảm chi phí.\n3. **Monitoring**: Cung cấp các công cụ giám sát và phân tích hiệu suất của cụm Kubernetes, giúp khách hàng nhanh chóng phát hiện và giải quyết vấn đề.\n4. **Logging**: Cung cấp các công cụ thu thập và phân tích log, giúp khách hàng theo dõi và phân tích hoạt động của ứng dụng.\n5. **Security**: Cung cấp các tính năng bảo mật như mạng lưới riêng ảo (VPC), nhóm bảo mật (Security Group) và xác thực truy cập (IAM).\n6. **High Availability**: Đảm bảo cụm Kubernetes luôn sẵn sàng và có thể chịu được sự cố, giúp khách hàng duy trì hoạt động kinh doanh liê

# Message persistence

In [19]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [20]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [21]:
config = {"configurable": {"thread_id": "abc123"}}

In [22]:
query = "Làm sao để dạy một con cá biết leo cây?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Đầu tiên, tôi muốn nói rằng cá không thể leo cây. Cá là động vật thủy sinh, chúng sống trong nước và không có khả năng di chuyển trên cạn hoặc leo cây. Cơ thể của cá được thiết kế để thích nghi với môi trường nước, với vây, đuôi và mang giúp chúng di chuyển và hô hấp trong nước.

Vì vậy, việc dạy một con cá biết leo cây là không thể. Cá không có khả năng thực hiện hành động này và cũng không cần phải làm như vậy vì chúng đã thích nghi rất tốt với môi trường nước.

Nếu bạn đang tìm cách để giúp cá hoặc các động vật khác di chuyển hoặc thực hiện hành động nào đó, tôi sẵn sàng giúp đỡ. Tuy nhiên, trong trường hợp này, việc dạy cá leo cây là không khả thi và không cần thiết.


# Async message

In [23]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

In [24]:
# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Đầu tiên, tôi muốn nói rằng cá không thể leo cây. Cá là động vật thủy sinh, chúng sống trong nước và không có khả năng di chuyển trên cạn hoặc leo cây. Cơ thể của cá được thiết kế để thích nghi với môi trường nước, với vây, đuôi và mang giúp chúng di chuyển và hô hấp trong nước.

Vì vậy, việc dạy một con cá biết leo cây là không thể. Cá không có khả năng thực hiện hành động này và cũng không cần phải làm như vậy vì chúng đã thích nghi rất tốt với môi trường nước.

Nếu bạn đang tìm cách để giúp cá hoặc các động vật khác di chuyển hoặc thực hiện hành động nào đó, tôi sẵn sàng giúp đỡ. Tuy nhiên, trong trường hợp này, việc dạy cá leo cây là không khả thi và không cần thiết.


# Prompt templates

In [25]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Bạn là một trợ lý ảo. Hãy trả lời các câu hỏi bằng hết khả năng của bạn bằng Tiếng Việt.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [26]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [27]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Nếu muốn nói chia tay khi đã hết yêu, làm sao để đối phương không tổn thương? (Vui lòng trả lời bằng tiếng Việt, tiếng anh tao ngu lắm)"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Khi muốn nói chia tay với người mà bạn đã hết yêu, việc khiến cho đối phương không tổn thương là rất khó khăn. Tuy nhiên, có một số cách để bạn có thể thực hiện cuộc trò chuyện này một cách nhẹ nhàng và tôn trọng hơn:

1. **Chọn thời điểm và địa điểm phù hợp**: Tìm một nơi yên tĩnh và riêng tư, nơi cả hai có thể nói chuyện mà không bị làm phiền. Tránh nói chia tay trong những tình huống công khai hoặc khi đối phương đang gặp áp lực hoặc căng thẳng.

2. **Thành thật nhưng tế nhị**: Hãy thành thật về cảm xúc của bạn, nhưng cũng cần phải tế nhị và tránh chỉ trích hoặc批 đánh giá tiêu cực về đối phương. Thay vào đó, hãy tập trung vào việc bạn cảm thấy mối quan hệ không còn phù hợp với bạn.

3. **Sử dụng "tôi" thay vì "bạn"**: Khi nói về lý do chia tay, hãy sử dụng câu bắt đầu bằng "tôi" để thể hiện rằng đây là cảm xúc và suy nghĩ của bạn, chứ không phải là lỗi của đối phương. Ví dụ: "Tôi cảm thấy chúng ta đã x

In [28]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hãy tóm gọn nội dung bài hát Em của ngày hôm qua của Sơn Tùng MTP"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

"Em Của Ngày Hôm Qua" là một bài hát nổi tiếng của Sơn Tùng M-TP, được phát hành vào năm 2015. Bài hát thuộc thể loại pop ballad và có giai điệu sâu lắng, xúc động.

Nội dung bài hát nói về nỗi nhớ và sự day dứt của người con trai khi phải chia tay với người yêu. Anh nhớ lại những kỷ niệm đẹp đẽ của hai người, từ những giây phút hạnh phúc cùng nhau cho đến những lúc chia ly đầy đau khổ.

Bài hát thể hiện cảm xúc sâu sắc và chân thành của Sơn Tùng M-TP, với giọng hát đầy cảm xúc và cách xử lý giai điệu tinh tế. Lời bài hát cũng rất dễ nhớ và gần gũi, giúp người nghe dễ dàng đồng cảm và kết nối với cảm xúc của nhân vật trong bài hát.

Tổng thể, "Em Của Ngày Hôm Qua" là một bài hát tình ca đẹp đẽ và đầy cảm xúc, thể hiện tài năng và khả năng sáng tác của Sơn Tùng M-TP. Bài hát đã trở thành một trong những hit lớn nhất của anh và được yêu thích rộng rãi bởi khán giả Việt Nam.


In [29]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Câu hỏi mà tôi vừa hỏi bạn là gì vậy"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Câu hỏi mà bạn vừa hỏi tôi là: "Hãy tóm gọn nội dung bài hát Em của ngày hôm qua của Sơn Tùng MTP"


# Using place holder

In [30]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [31]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [32]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Làm sao để nói chia tay nhưng đối phương phải là người chủ động nói"
language = "Vietnamese"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Để đạt được mục tiêu này, bạn cần sử dụng một số chiến thuật giao tiếp tinh tế và tạo ra một môi trường mà trong đó, đối phương cảm thấy tự nhiên khi đưa ra quyết định kết thúc mối quan hệ. Dưới đây là một số bước bạn có thể thực hiện:

1. **Tạo khoảng cách**: Bắt đầu bằng cách giảm dần tần suất và chất lượng của các cuộc gặp gỡ hoặc trao đổi thông tin. Điều này giúp đối phương cảm nhận được sự thay đổi trong mối quan hệ mà không cần phải nói rõ ràng.

2. **Thay đổi cách giao tiếp**: Thay đổi cách bạn phản hồi hoặc tương tác với họ. Ví dụ, nếu trước đây bạn luôn trả lời nhanh chóng và chi tiết, hãy thử chậm lại và ngắn gọn hơn. Điều này có thể khiến đối phương cảm thấy không còn được quan tâm như trước.

3. **Tập trung vào bản thân**: Hãy cho đối phương biết rằng bạn đang tập trung vào các khía cạnh khác của cuộc sống, như công việc, học tập, hoặc sở thích cá nhân. Điều này có thể giúp họ nhận ra rằng bạn

# Managing Conversation History


In [33]:
trimmer = trim_messages(
    max_tokens=1000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [34]:
messages = [
    SystemMessage(content="Bạn là một trợ lý ảo"),
    HumanMessage(content="Chào mày, tao là Cường"),
    AIMessage(content="Chào bạn."),
    HumanMessage(content="Cho tao biết, VKS là gì vậy?"),
    AIMessage(
        content="VKS là viết tắt của VngCloud Kubernetes Service, một dịch vụ của VngCloud nhằm cung cấp các cụm Kubernetes đến khách hàng. VKS cung cấp các tính năng như fully managed Kubernetes, autoscaling, monitoring, logging, và nhiều tính năng khác."
    ),
    HumanMessage(content="VKS có hỗ trợ CSI không?"),
    AIMessage(
        content="Có chứ, CSI là viết tắt của Container Storage Interface, một chuẩn giao diện cho phép các hệ thống lưu trữ nền tảng cung cấp dịch vụ lưu trữ cho các ứng dụng chạy trên Kubernetes."
    ),
    HumanMessage(content="Làm sao để tạo một PersistentVolumeClaim trên VKS?"),
    AIMessage(
        content="""Đây là một ví dụ để tạo một PersistentVolumeClaim 20Gi trên VKS\n```yaml
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: my-expansion-storage-class                    # [1] The StorageClass name, CAN be changed
provisioner: bs.csi.vngcloud.vn                       # The VNG-CLOUD CSI driver name
parameters:
  type: vtype-61c3fc5b-f4e9-45b4-8957-8aa7b6029018    # The volume type UUID
allowVolumeExpansion: true                            # MUST set this value to turn on volume expansion feature
---

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-expansion-pvc                           # [2] The PVC name, CAN be changed
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 20Gi                                # [3] The PVC size, CAN be changed, this value MUST be in the valid range of the proper volume type
  storageClassName: my-expansion-storage-class 
```"""
    ),
    HumanMessage(content="Vậy volume 100Gi thì sao?"),
    AIMessage(content="yes!"),
]

In [36]:
trimmer.invoke(messages)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


[SystemMessage(content='Bạn là một trợ lý ảo', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Chào mày, tao là Cường', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Chào bạn.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Cho tao biết, VKS là gì vậy?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='VKS là viết tắt của VngCloud Kubernetes Service, một dịch vụ của VngCloud nhằm cung cấp các cụm Kubernetes đến khách hàng. VKS cung cấp các tính năng như fully managed Kubernetes, autoscaling, monitoring, logging, và nhiều tính năng khác.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='VKS có hỗ trợ CSI không?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Có chứ, CSI là viết tắt của Container Storage Interface, một chuẩn giao diện cho phép các hệ thống lưu trữ nền tảng cung cấp dịch vụ lưu trữ cho các ứng dụng chạy trên Kubernetes.', additional_kwargs={}, response_metad

In [37]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    # trimmed_messages = trimmer.invoke(state["messages"])
    trimmed_messages = state["messages"]
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [38]:
config = {"configurable": {"thread_id": "abc567"}}
query = "VKS là gì?"
language = "Vietnamese"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

VKS là viết tắt của Viện Kiểm sát, một cơ quan nhà nước có chức năng kiểm sát hoạt động tư pháp ở Việt Nam. VKS có nhiệm vụ kiểm sát việc tuân thủ pháp luật trong hoạt động điều tra, truy tố, xét xử và thi hành án.

Tuy nhiên, trong ngữ cảnh trước đó, tôi đã giải thích rằng VKS là viết tắt của VngCloud Kubernetes Service, một dịch vụ của VngCloud nhằm cung cấp các cụm Kubernetes đến khách hàng.


In [39]:
config = {"configurable": {"thread_id": "abc567"}}
query = "Tạo PersistentVolumeClaim 100Gi trên VKS"
language = "Vietnamese"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Để tạo một PersistentVolumeClaim 100Gi trên VKS, bạn có thể sử dụng file YAML sau:
```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-pvc-100gi
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 100Gi
  storageClassName: standard
```
Sau đó, bạn có thể áp dụng file YAML này vào cluster của mình bằng cách chạy lệnh:
```
kubectl apply -f pvc-100gi.yaml
```
Lưu ý rằng bạn cần thay thế `standard` bằng tên của StorageClass mà bạn muốn sử dụng. Nếu bạn không biết tên của StorageClass, bạn có thể xem danh sách các StorageClass có sẵn trên cluster của mình bằng cách chạy lệnh:
```
kubectl get storageclass
```


In [40]:
config = {"configurable": {"thread_id": "abc678"}}
query = "Câu hỏi mà tôi vừa hỏi bạn là gì vậy"
language = "Vietnamese"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Bạn vừa hỏi về việc tạo một PersistentVolumeClaim với kích thước 100Gi trên VKS.


# Streaming

In [41]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Cho tôi xin lời bài hát Em của ngày hôm qua của Sơn Tùng MTP (đoạn rap)"
language = "Vietnamese"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

Tôi| không| thể| cung| cấp| lời| bài| hát| "|Em| C|ủa| Ngày| H|ôm| Qu|a|"| của| Sơn| T|ùng| M|-|TP| vì| lý| do| bản| quyền|.| Tuy| nhiên|,| tôi| có| thể| giúp| bạn| tìm| kiếm| lời| bài| hát| này| trên| các| nền| tảng| âm| nhạc| hoặc| trang| web| chính| thức| của| nghệ| sĩ|.||